In [1]:
import json
from tqdm import tqdm
from glob import glob
import pandas as pd
import numpy as np
from natsort import natsorted
import torch
import torch.nn as nn
import re
import os

data_path = './morpheme/10'

data_10 = natsorted(glob(data_path + '/*.json'))

# load 전처리 데이터 세트
use_data = pd.read_csv('./4주차_단어정리.csv',encoding='cp949')[['최종단어']].dropna().최종단어.values

In [3]:
# 원래 numpy 버전 1.24.4 -> 1.21.1

In [2]:
def collect_word(path):
    collected_word = []
    for file_path in tqdm(path):
        with open(file_path,'r',encoding = 'utf-8') as file:
            json_file = json.load(file)
            
            try:
                collected_word.append(json_file['data'][0]['attributes'][0]['name'])
            except:
                print(json_file['data'],'File is blank')
                pass
            
    return set(collected_word)


In [3]:
data_10_words = collect_word(data_10)

100%|██████████| 15000/15000 [24:38<00:00, 10.14it/s] 


In [4]:
print(f'Data_10 Length : {len(data_10_words)}')

Data_10 Length : 2774


- 일상에서 사용하는 단어들도 있는 거 같긴 한데, 지명 관련된 단어들이 굉장히 많다.

In [6]:
# all_data = data_10_words.union(data_11_words)
np.save('AI_hub_10_word_list.npy',np.array(list(data_10_words)))
len(data_10_words)

2774

In [2]:
all_data = np.load('./AI_hub_10_word_list.npy')
all_data

array(['호수', '견습소', '연수구주민지원센터', ..., '햇빛', '보은군지부', '제발'], dtype='<U12')

#### 

대충 걸러봤을 때 800개 정도의 단어 샘플들이 존재하는 거 같다.

- 그렇다면 20명의 수어 전문가가 5개의 각도에서 촬영을 진행했으므로 하나의 라벨당 100개의 sample이 존재하는 것임.

### Data Preprocessing

- 10번 파일만을 우선 사용

In [3]:
def collect_file_name(path,filtered_word):
    collected_file = {}
    for file_path in tqdm(path):
        with open(file_path,'r',encoding = 'utf-8') as file:
            json_file = json.load(file)
            file_name = file_path[file_path.rindex('NIA'):file_path.rindex('_morpheme')]
            
            try:
                word = json_file['data'][0]['attributes'][0]['name']
                if word in filtered_word and file_name[-1] == "F": # Front만 사용
                    collected_file[file_name] = word
            except:
                print(json_file['data'],'File is blank')
                pass
            
    return collected_file

In [4]:
# 파일명 : 단어 형태의 딕셔너리 생성
collected_file = collect_file_name(data_10,use_data)

100%|██████████| 15000/15000 [03:57<00:00, 63.29it/s] 


In [6]:
# 동일한 영상 중에서 중복되는 영상이 존재하는 것으로 판단됨. 우선 상관없이 진행해보도록 함.
len(collected_file)

820

In [11]:
collected_file

{'NIA_SL_WORD0001_REAL10_F': '고민',
 'NIA_SL_WORD0005_REAL10_F': '눈',
 'NIA_SL_WORD0007_REAL10_F': '음료수',
 'NIA_SL_WORD0008_REAL10_F': '발가락',
 'NIA_SL_WORD0009_REAL10_F': '슬프다',
 'NIA_SL_WORD0010_REAL10_F': '자극',
 'NIA_SL_WORD0011_REAL10_F': '안타깝다',
 'NIA_SL_WORD0012_REAL10_F': '어색하다',
 'NIA_SL_WORD0014_REAL10_F': '외국인',
 'NIA_SL_WORD0016_REAL10_F': '신사',
 'NIA_SL_WORD0020_REAL10_F': '장애인',
 'NIA_SL_WORD0025_REAL10_F': '싫어하다',
 'NIA_SL_WORD0026_REAL10_F': '급하다',
 'NIA_SL_WORD0029_REAL10_F': '검사',
 'NIA_SL_WORD0034_REAL10_F': '독서',
 'NIA_SL_WORD0037_REAL10_F': '감기',
 'NIA_SL_WORD0038_REAL10_F': '배드민턴',
 'NIA_SL_WORD0043_REAL10_F': '불행',
 'NIA_SL_WORD0045_REAL10_F': '사위',
 'NIA_SL_WORD0046_REAL10_F': '설사',
 'NIA_SL_WORD0050_REAL10_F': '손녀',
 'NIA_SL_WORD0051_REAL10_F': '손자',
 'NIA_SL_WORD0055_REAL10_F': '예식장',
 'NIA_SL_WORD0057_REAL10_F': '회복',
 'NIA_SL_WORD0060_REAL10_F': '입원',
 'NIA_SL_WORD0063_REAL10_F': '축구장',
 'NIA_SL_WORD0064_REAL10_F': '치료',
 'NIA_SL_WORD0065_REAL10_F': '치료법',
 'NI

In [20]:
np.save('folder_tae.npy', np.array(list(collected_file.keys()))) # 사용한 폴더 
np.save('label_tae.npy',np.array(list(collected_file.values()))) # 사용한 라벨

In [21]:
# 위에서 추린 Dictionary 파일을 넣기
def filtering_using_folder(filtered_folder_name): 
    using_keypoint_list = []

    for folder in tqdm(filtered_folder_name.keys()):
        if glob(f'./keypoints/10/{folder}/*.json') == []: # 만약 폴더 명이 잘못 된게 있다면
            print(folder)
        else:
            using_keypoint_list.append(natsorted(glob(f'./keypoints/10/{folder}/*.json')))


    # using_keypoint_list = [sorted(glob(f'./keypoints/10/{folder}/*.json')) for folder in tqdm(filtered_folder_name.keys())]
    return using_keypoint_list

# 사용할 keypoint 파일 목록
keypoint_file_list = filtering_using_folder(collected_file)

100%|██████████| 820/820 [00:03<00:00, 246.52it/s]


NIA_SL_WORD1536_REAL10_U_F -> NIA_SL_WORD1536_REAL10_F  
NIA_SL_WORD0681_REAL10_U_F -> NIA_SL_WORD0681_REAL10_F

In [22]:
## Keypoint 값들을 불러오기
def collect_keypoints(path_list):
    keypoint_sequence = []
    for file_paths in tqdm(path_list):
        folder_res = []
        for file_path in file_paths:
            sequence_res = []
            with open(file_path,'r',encoding = 'utf-8') as file:
                json_file = json.load(file)
                # 각 keypoint 값들을 합치기
                for key in list(json_file['people'].keys())[1:5]: # face_keypoints_2d, pose_keypoints_2d, hand_left_keypoints_2d, hand_right_keypoints_2d
                    for idx,keypoint in enumerate(json_file['people'][key],start=1):
                        # confidence 값 버리기
                        if idx % 3 != 0:
                            sequence_res.append(keypoint)
                            
            # folder_result에 추가
            folder_res.append(sequence_res)
            
        keypoint_sequence.append(folder_res)
    return keypoint_sequence

keypoint_sequence_by_folder = collect_keypoints(keypoint_file_list)

100%|██████████| 820/820 [23:20<00:00,  1.71s/it]


In [23]:
## save
np.save('./keypoint_10_sequence_by_folder.npy',np.array(keypoint_sequence_by_folder))
# 행마다 길이가 달라서 나타나는 warning

/tmp/ipykernel_656317/2919097853.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save('./keypoint_10_sequence_by_folder.npy',np.array(keypoint_sequence_by_folder))


In [4]:
# 불러오기
keypoint_sequence_by_folder = np.load('./keypoint_10_sequence_by_folder.npy', allow_pickle=True)

In [5]:
np.array(keypoint_sequence_by_folder).shape

(820,)

In [6]:
print(np.array(keypoint_sequence_by_folder[0]).shape)
print(np.array(keypoint_sequence_by_folder[1]).shape)
print(np.array(keypoint_sequence_by_folder[2]).shape)
print(np.array(keypoint_sequence_by_folder[3]).shape)

(132, 274)
(146, 274)
(146, 274)
(133, 274)


In [27]:
## 무게중심을 기준으로 정규화를 진행해줘야함.
## 정규화 진행하지 않을 시 학습이 안된다고 선행논문에서 언급이 되어있음.
## 좌측 어깨 72번째 좌표, 우측 75번째 좌표의 평균으로 대체
def normalize_keypoint(keypoint_sequences):
    
    normalized_keypoints_result = []

    for fold in tqdm(keypoint_sequences):
        normalized_fold_keypoints = []
        for frame in fold:
            x = (frame[142],frame[148])
            y = (frame[143],frame[149])
            x_mean, x_std = np.mean(x), np.std(x)
            y_mean, y_std = np.mean(y), np.std(y)
            
            normalized_values = []
            # 정규화 진행
            for idx,value in enumerate(frame):
                if idx % 2 == 0:
                    normalized_values.append(round((value - x_mean)/x_std, 5)) # round를 치는게 좋아보임
                else:
                    normalized_values.append(round((value - y_mean)/y_std, 5))
                    
            # 각 프레임마다 해당 folder로 추가
            normalized_fold_keypoints.append(normalized_values)
            
        normalized_keypoints_result.append(normalized_fold_keypoints)

    return normalized_keypoints_result

normalized_keypoints_by_folder = normalize_keypoint(keypoint_sequence_by_folder)

# save 
np.save('./normalized_keypoints_10_by_folder.npy',np.array(normalized_keypoints_by_folder))

 19%|█▉        | 159/820 [00:27<01:47,  6.15it/s]/tmp/ipykernel_656317/3457229548.py:20: RuntimeWarning: divide by zero encountered in double_scalars
  normalized_values.append(round((value - x_mean)/x_std, 5)) # round를 치는게 좋아보임
/tmp/ipykernel_656317/3457229548.py:20: RuntimeWarning: invalid value encountered in double_scalars
  normalized_values.append(round((value - x_mean)/x_std, 5)) # round를 치는게 좋아보임
 27%|██▋       | 220/820 [00:37<01:53,  5.28it/s]/tmp/ipykernel_656317/3457229548.py:22: RuntimeWarning: divide by zero encountered in double_scalars
  normalized_values.append(round((value - y_mean)/y_std, 5))
/tmp/ipykernel_656317/3457229548.py:22: RuntimeWarning: invalid value encountered in double_scalars
  normalized_values.append(round((value - y_mean)/y_std, 5))
100%|██████████| 820/820 [02:22<00:00,  5.76it/s]
/tmp/ipykernel_656317/3457229548.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarra

In [7]:
# 불러오기
normalized_keypoints_by_folder = np.load('./normalized_keypoints_10_by_folder.npy', allow_pickle=True)

In [8]:
np.array(normalized_keypoints_by_folder).shape

(820,)

### Modeling 

In [1]:
import json
from tqdm.auto import tqdm
from glob import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import re
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim

/root/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# data load
data = np.load('./normalized_keypoints_10_by_folder.npy',allow_pickle = True)
# data

# label load 
label = np.load('./label_10.npy') # label, 820
label

array(['고민', '눈', '음료수', '발가락', '슬프다', '자극', '안타깝다', '어색하다', '외국인', '신사',
       '장애인', '싫어하다', '급하다', '검사', '독서', '감기', '배드민턴', '불행', '사위', '설사',
       '손녀', '손자', '예식장', '회복', '입원', '축구장', '치료', '치료법', '퇴원', '고추',
       '냄비', '찬물', '돼지고기', '두부', '딸기', '떡국', '라면', '막걸리', '무', '밥그릇',
       '밥솥', '부엌', '비빔밥', '사과', '달다', '소주', '술잔', '식당', '양식', '술', '참외',
       '칼국수', '커피', '콜라', '와인', '운동화', '잠옷', '이불', '에어컨', '월세', '전세',
       '치약', '가수', '경고', '경찰서', '공식', '과장', '관광버스', '교수', '교장', '독일어',
       '동아리', '디자이너', '사무', '여행사', '예술가', '외국어', '요리사', '의사', '이력서',
       '자가용', '주차장', '출근', '코치', '터미널', '근무', '판결', '평가', '희생', '간호사',
       '논', '대기업', '백화점', '수입', '아르바이트', '고등학교', '고등학생', '교실', '교육자',
       '예절', '복습', '분필', '서점', '실습', '여학생', '연구소', '연구실', '예습', '유학',
       '의학', '지도', '재학', '연구', '학부모', '훈련', '경기도', '남산', '울산', '일요일',
       '축하', '장난', '장인', '디자인', '동화책', '문학', '음악가', '예술', '초상화', '공군',
       '감옥', '국왕', '강', '달빛', '장마', '체온', '파도', '하늘', '강아지', '꽃씨', '꽃잎',
     

In [11]:
# 가장 sequence가 긴 데이터 찾기
result = 0
for idx, ele in enumerate(data):
    if result <= len(ele):
        result = len(ele)
print(f'최대 sequence 길이 : {result}')

최대 sequence 길이 : 197


In [12]:
np.array(data[0]).shape[0]

132

In [17]:
np.array([[0]*10])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [23]:
np.array(data[0]).shape

(132, 274)

In [25]:
np.array(data[0])[:10, :].shape

(10, 274)

In [31]:
np.zeros([10, 274])

(10, 274)

In [32]:
# padding sequence는 197로 진행하도록 함.
# 테스트를 위해 244로 만듦
def padding_sequence(seq, length=244):

    original_array = np.array(seq)
    original_shape = original_array.shape[0]
    repeat = length - original_shape # 최대 길이보다 짧아서 반복해서 입력해야 하는 횟수
    pad_iter = (repeat // original_shape) + 1 

    if pad_iter >=2: # 길이가 2배를 넘을 때
        for i in range(pad_iter):
            # original_array = np.vstack((original_array,original_array[:repeat,:])) # 반복해야하는만큼 앞을 채워줌
            original_array = np.vstack((original_array,np.zeros([repeat, 274]))) # 반복해야하는만큼 앞을 채워줌
            if repeat - original_shape <0:
                repeat %= original_shape
            else:
                repeat -= original_shape
            
            if original_array.shape[0] == length:
                return original_array
    else:
        return np.vstack((original_array,original_array[:repeat,:]))

padding_data = []
for i, value in tqdm(enumerate(data)):
    try:
        padding_data.append(padding_sequence(value))
    except:
        print(i)

820it [00:01, 618.24it/s]


In [33]:
len(padding_data)

820

In [34]:
np.array(padding_data).shape

(820, 244, 274)

In [38]:
padding_data[0].shape

(244, 274)

In [39]:
np.save('./padding_data_10_244_zero.npy',padding_data)

### Data Load

In [3]:
# data load
data = np.load('./padding_data_10_244_zero.npy')
# data

# label load 
label = np.load('./label_10.npy')

In [4]:
# null값이 들어있는 sequence가 있어서 제거 진행.
sorted_notnull_data = []
sorted_notnull_label = []
for idx,i in enumerate(data):
    has_nan = np.isnan(i).any()
    if has_nan:
#         nan_indices = np.where(np.isnan(i))[0]
#         nan_indices
        print("배열에 NaN 값이 포함되어 있습니다.")
        print(idx)
    else:
        sorted_notnull_data.append(i)
        sorted_notnull_label.append(label[idx])

배열에 NaN 값이 포함되어 있습니다.
159
배열에 NaN 값이 포함되어 있습니다.
220
배열에 NaN 값이 포함되어 있습니다.
380
배열에 NaN 값이 포함되어 있습니다.
658


In [5]:
label[159]

'최대'

In [6]:
len(sorted_notnull_data)

816

In [7]:
class CustomDataset(Dataset): 
    def __init__(self,data,label):
        self.x_data = data
        self.y_data = [i for i in range(len(label))]

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)

    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.tensor(self.y_data[idx]).long()
        return x, y

In [8]:
np.unique(label).shape

(707,)

In [9]:
num_epochs = 200
seq_len = 253 # 변경
input_size = 274
num_classes = 816 # 변경
hidden_size = 198
num_layers = 2
lr = 0.001
batch_size = 8

In [10]:
dataset = CustomDataset(sorted_notnull_data,sorted_notnull_label)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [11]:
criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [12]:
device

device(type='cuda', index=0)

In [13]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers):
        super(LSTMModel, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size,512)
        self.fc2 = nn.Linear(512,num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.shape[0], hidden_size).to(device) 
        c0 = torch.zeros(num_layers, x.shape[0], hidden_size).to(device)
        out, _ = self.lstm(x,(h0,c0))
        out = self.fc1(out[:, -1, :])  # 마지막 타임 스텝의 출력만 사용
        F.relu(out)
        out = self.fc2(out)
        return out

model = LSTMModel(input_size, hidden_size, num_classes, num_layers).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

print(model)

LSTMModel(
  (lstm): LSTM(274, 198, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=198, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=816, bias=True)
)


In [14]:
# accuracy 계산 함수 생성
def cal_accuracy(outputs,labels):
    count = 0 
    for idx,output in enumerate(outputs):
        if output == labels[idx]:
            count += 1
            
    return count

In [15]:
# 학습 진행
best_loss = 1e9
best_accuracy = 0.0
train_losses = []
train_accuracies = []
result_epoch = 0

for epoch in range(num_epochs):
    print(f'------{epoch + 1}번째 에폭 학습 시작------')
    training_accuracy = 0
    training_loss = 0.0
    
    for i, (sequences, labels) in enumerate(tqdm(dataloader)):
        sequences = sequences.to(device) 
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        training_accuracy += cal_accuracy(torch.argmax(outputs,dim=1),labels)
        training_loss += loss.item()
        
        torch.cuda.empty_cache()
        
        # if i % 100 == 0:
            # print(f'Mini Batch {i}, Train Loss {loss.item():.4f}')
    
    training_loss /= len(dataloader)
    training_accuracy = training_accuracy / (len(dataloader)*batch_size) *100
    
    train_losses.append(training_loss)
    train_accuracies.append(training_accuracy)
    
    if best_loss >= training_loss and best_accuracy <= training_accuracy: 
        torch.save(model.state_dict(),'./checkpoint/keypoint_10_244_lstm.pth')
        best_loss = training_loss
        best_accuracy = training_accuracy
        result_epoch= epoch
        print(f'--Best Training Loss : {best_loss:.4f} & Best Training Accuracy: {best_accuracy:.4f}! Model SAVE!--')

    print('---Epoch 학습 결과 ---')
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {training_loss:.4f} \n Train Accuracy: {training_accuracy:.4f}%')

------1번째 에폭 학습 시작------


100%|██████████| 102/102 [00:02<00:00, 36.34it/s]


--Best Training Loss : 6.5721 & Best Training Accuracy: 0.0000! Model SAVE!--
---Epoch 학습 결과 ---
Epoch [1/200], Train Loss: 6.5721 
 Train Accuracy: 0.0000%
------2번째 에폭 학습 시작------


100%|██████████| 102/102 [00:02<00:00, 34.40it/s]


---Epoch 학습 결과 ---
Epoch [2/200], Train Loss: 6.9828 
 Train Accuracy: 0.0000%
------3번째 에폭 학습 시작------


...

---Epoch 학습 결과 ---
Epoch [198/200], Train Loss: 2.4940 
 Train Accuracy: 57.7206%
------199번째 에폭 학습 시작------


100%|██████████| 102/102 [00:02<00:00, 50.74it/s]


--Best Training Loss : 2.4942 & Best Training Accuracy: 57.8431! Model SAVE!--
---Epoch 학습 결과 ---
Epoch [199/200], Train Loss: 2.4942 
 Train Accuracy: 57.8431%
------200번째 에폭 학습 시작------


100%|██████████| 102/102 [00:01<00:00, 52.23it/s]

---Epoch 학습 결과 ---
Epoch [200/200], Train Loss: 2.4925 
 Train Accuracy: 57.7206%


In [16]:
# Final Loss and Accuracy
print(f'Final Resut Epoch : {result_epoch} Best Accuracy : {max(train_accuracies)} & Best Loss : {min(train_losses)} ')

Final Resut Epoch : 198 Best Accuracy : 57.84313725490197 & Best Loss : 2.492477658254346 


## Test

In [17]:
# data load
test_data = np.load('./padding_data_01_244.npy')

# label load 
test_label = np.load('./label_01.npy')

In [18]:
test_data.shape, test_label.shape

((820, 244, 274), (820,))

In [19]:
# null값이 들어있는 sequence가 있어서 제거 진행.
sorted_notnull_test_data = []
sorted_notnull_test_label = []
for idx,i in enumerate(test_data):
    has_nan = np.isnan(i).any()
    if has_nan:
#         nan_indices = np.where(np.isnan(i))[0]
#         nan_indices
        print("배열에 NaN 값이 포함되어 있습니다.")
        print(idx)
    else:
        sorted_notnull_test_data.append(i)
        sorted_notnull_test_label.append(test_label[idx])

In [20]:
num_epochs = 200
seq_len = 244 # 변경
input_size = 274
num_classes = 816 # 변경
hidden_size = 198
num_layers = 2
lr = 0.001
batch_size = 8

In [22]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers):
        super(LSTMModel, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size,512)
        self.fc2 = nn.Linear(512,num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.shape[0], hidden_size).to(device) 
        c0 = torch.zeros(num_layers, x.shape[0], hidden_size).to(device)
        out, _ = self.lstm(x,(h0,c0))
        out = self.fc1(out[:, -1, :])  # 마지막 타임 스텝의 출력만 사용
        F.relu(out)
        out = self.fc2(out)
        return out

test_model = LSTMModel(input_size, hidden_size, num_classes, num_layers).to(device)
# optimizer = optim.Adam(test_model.parameters(), lr=lr)
checkpoint = torch.load('./checkpoint/keypoint_10_244_zero_lstm.pth')
test_model.load_state_dict(checkpoint)
print(test_model)

LSTMModel(
  (lstm): LSTM(274, 198, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=198, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=816, bias=True)
)


In [23]:
test_dataset = CustomDataset(sorted_notnull_test_data, sorted_notnull_test_label)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [24]:
test_model.eval()

with torch.no_grad():
    correct = 0
    total = 0

    for batch_data, batch_labels in test_dataloader:
        # 모델 입력 데이터와 레이블을 GPU 또는 CPU로 이동시킵니다.
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)

        # 모델에 데이터를 전달하여 예측합니다.
        outputs = test_model(batch_data)

        # 확률 분포에서 최댓값을 가지는 클래스를 선택합니다.
        _, predicted = torch.max(outputs, 1)

        # 정확하게 분류된 샘플 수를 업데이트합니다.
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {100 * accuracy:.2f}%')

Accuracy: 0.73%
